# Imports 

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib
#import urllib3.request,sys,time
from urllib.error import HTTPError
from urllib.request import urlopen
import pandas as pd
import time
import warnings 
warnings.filterwarnings('ignore')

# Fonctions getArticle et getNews

In [2]:
def getArticle(pageurl):
    try:
        html = urlopen(pageurl)
    except HTTPError as e:
        return None
    try:
        bsObj = BeautifulSoup(html.read())
        item = bsObj.find("div",{'class':{'sch-results'}}).findAll('h3')
    except AttributeError as e:
        return None
    articles_list=[]
    for j in item:#one item is one article
        article=dict()
        article['url']="www.dailymail.co.uk"+j.find('a').get('href')
        articles_list.append(article)
    return articles_list

In [3]:
url = 'https://www.dailymail.co.uk/home/search.html?offset=0&size=50&sel=site&searchPhrase=Facebook+&sort=recent&type=article&type=video&type=permabox&days=all'

In [4]:
html = urlopen(url)
bsObj = BeautifulSoup(html.read())
item = bsObj.find("div",{'class':{'sch-results'}}).findAll('h3')

In [ ]:
bsObj

In [ ]:
item

In [7]:
type(bsObj)

bs4.BeautifulSoup

In [ ]:
getArticle(url)

In [9]:
#crée un dictionnaire avec les infos de l'article
def getNews(article_url):
    try:
        html = urlopen(article_url)
    except HTTPError as e:
        print(article_url)
        return None
    try :
        bsObj = BeautifulSoup(html.read())
        article=dict()
    except :
        return None
    try:
        article["author"]=bsObj.find("a",{'class':{'author'}}).text
    except AttributeError as e:
        article["author"]=''
    try:
        article["headline"]=bsObj.find("div",{'id':{'js-article-text'}}).h2.text
    except AttributeError as e:
        article["headline"]=''
    try:
        article["summary"]=bsObj.find("ul",{'class':{'mol-bullets-with-font'}}).text
    except AttributeError as e:
        article["summary"]=''
    try:
        article["date"]=bsObj.find("span",{'class':{'article-timestamp article-timestamp-updated'}}).time["datetime"][:10]
    except AttributeError as e:
        article["date"]= ''
    try:
        paras=bsObj.find_all("div",{'itemprop':{'articleBody'}})
        fulltext=""
        for para in paras:
            paras_text=" ".join(i.text for i in para.find_all("p", {'class':'mol-para-with-font'}))
            fulltext=fulltext+paras_text
        article["fulltext"]=fulltext
    except AttributeError as e:
        article["fulltext"]=''
    article["url"]=article_url
    time.sleep(1)
    return article

In [ ]:
articles_list = []

In [35]:
#transforme la liste d'url en dataframe avec les articles
def get_articles_from_links(articles_links):
    articles = []
    for ind in articles_links.index : 
        if ind%1000==0:
            print(ind)
        try :
            article_url='https://'+articles_links.iloc[ind,0]
            news=getNews(article_url)
            articles_list.append(news)
            if news is not None:
                articles.append(news)
        except :
            return None
        time.sleep(1)
    return articles

In [36]:
def articles_from_base_url_DM(base_url1, base_url2):
    url = []
    for i in range (0, 14400, 50): #les pages comprennent 50 articles, donc on parcourt par 50. Le rangemax dépend du sujet. 14400 pour Facebook. 
        url.append(base_url1+str(i)+base_url2)
    articles_links = pd.DataFrame()
    #création de la database avec tous les liens d'articles
    for i in range(len(url)):
        inter = pd.DataFrame(getArticle(url[i]))
        articles_links = pd.concat([articles_links, inter], ignore_index=True, axis=0)
    news = get_articles_from_links(articles_links)
    df_news = pd.DataFrame(news)
    return df_news    

# Tries with Facebook

In [12]:
#URL de base 
base_url1 = 'https://www.dailymail.co.uk/home/search.html?offset='
base_url2 = '&size=50&sel=site&searchPhrase=Facebook+&sort=recent&type=article&topic=Facebook&topic=Facebook+%28Old%29&days=all'

In [13]:
#parcours de tous les articles dispos 
fb_url = []
for i in range (0, 100, 50): #les pages comprennent 50 articles, donc on parcourt par 50. Le rangemax dépend du sujet. 
    fb_url.append(base_url1+str(i)+base_url2)

In [14]:
articles_links = pd.DataFrame()

In [15]:
#création de la database avec tous les liens d'articles
for i in range(len(fb_url)):
    inter = pd.DataFrame(getArticle(fb_url[i]))
    articles_links = pd.concat([articles_links, inter], ignore_index=True, axis=0)

In [16]:
articles_links

,url
0,www.dailymail.co.uk/femail/article-12311019/Br...
1,www.dailymail.co.uk/femail/food/article-123092...
2,www.dailymail.co.uk/femail/real-life/article-1...
3,www.dailymail.co.uk/news/article-12308207/LAPD...
4,www.dailymail.co.uk/femail/article-12308191/Iv...
5,www.dailymail.co.uk/news/article-12306401/Moth...
6,www.dailymail.co.uk/news/article-12305749/Geel...
7,www.dailymail.co.uk/femail/real-life/article-1...
8,www.dailymail.co.uk/femail/cleaning/article-12...
9,www.dailymail.co.uk/news/article-12305265/Bris...


In [17]:
news = get_articles_from_links(articles_links)

0


In [18]:
df_news = pd.DataFrame(news)

Même problème qu'avec le DE, trop d'articles. On ne garde que ceux dont l'url concerne directement facebook. 

In [18]:
df_fb_DM = df_news[df_news['url'].str.contains('Facebook|Meta|facebook')]

Comme pour le DE, on définit une fonction qui systématise la traitement.

# Dataframe Facebook

In [20]:
articles_list = []

In [21]:
#URL de base 
base_url1 = 'https://www.dailymail.co.uk/home/search.html?offset='
base_url2 = '&size=50&sel=site&searchPhrase=Facebook+&sort=recent&type=article&topic=Facebook&topic=Facebook+%28Old%29&days=all'

In [22]:
df_news_fb = articles_from_base_url_DM(base_url1, base_url2)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


In [28]:
df_news_fb['url'] = df_news_fb['url'].str.lower()

In [29]:
df_news_fb['url'][123]

'https://www.dailymail.co.uk/sport/mma/article-12238811/elon-musk-accepts-incredible-offer-ufc-legend-georges-st-pierre-mark-zuckerberg-fight.html'

In [23]:
len(articles_list)

0

In [ ]:
with open("liste_temporaire_1_fb_DM.json", "w") as fp:
  json.dump(articles_list, fp)

In [30]:
df_fb_DM = df_news_fb[df_news_fb['url'].str.contains('meta|facebook|messenger|instagram|oculus|whatsApp|zuckerberg|olivan|clegg|social-media|gafa|gafam|big-tech|consumer-privacy|tech')]

In [32]:
df_fb_DM.to_csv('df_fb_DM.csv')

# Dataframe Amazon 

In [37]:
articles_list = []

In [38]:
#URL de base 
base_url1 = 'https://www.dailymail.co.uk/home/search.html?offset='
base_url2 = '&size=50&sel=site&searchPhrase=Amazon&sort=recent&type=article&topic=Amazon+%28Old%29&topic=Amazon&days=all'

In [39]:
df_news_am = articles_from_base_url_DM(base_url1, base_url2)

0
1000
https://www.dailymail.co.uk/sciencetech/article-3497401/Is-Amazon-set-buy-Google-s-humanoid-robot-division-Boston-Dynamics-sale-Atlas-used-warehouses-robot-dogs-making-deliveries.html
https://www.dailymail.co.uk/sciencetech/article-3492903/Pay-WINK-Amazon-patents-uses-selfies-blinking-pay-online.html
https://www.dailymail.co.uk/news/article-3487857/Amazon-shows-stories-workers-sacked-stealing-big-screens-warehouses-deter-employees-taking-packages.html
2000


In [41]:
df_news_am['url'] = df_news_am['url'].str.lower()

In [42]:
df_am_DM = df_news_am[df_news_am['url'].str.contains('amazon|prime|whole-foods|zappos|pillpack|twitch|audible|goodreads|imdb|bezos|jassy|olsavsky|alexander|gafa|gafam|big-tech|consumer-privacy|tech')]

In [43]:
yy=df_am_DM[df_am_DM['fulltext'].str.contains('forest|tribe|jungle|swallows-and-amazon|amazons')]
df_am_DM.drop(yy.index,inplace=True)

In [51]:
df_am_DM.to_csv('df_am_DM.csv')

# Dataframe Apple

In [46]:
#URL de base 
base_url1 = 'https://www.dailymail.co.uk/home/search.html?offset='
base_url2 = '&size=50&sel=site&searchPhrase=Apple&sort=recent&type=article&topic=Apple+%28Old%29&topic=Apple&days=all'

In [47]:
df_news_ap = articles_from_base_url_DM(base_url1, base_url2)

0
1000
2000
3000
4000
5000


In [48]:
df_news_ap['url'] = df_news_ap['url'].str.lower()

In [49]:
df_ap_DM = df_news_ap[df_news_ap['url'].str.contains('apple|iphone|ipad|mac|watch|macbook|ios|airpods|app-store|itunes|icloud|beats|siri|shazam|cook|jobs|gafa|gafam|big-tech|consumer-privacy|tech')]

In [50]:
yy=df_ap_DM[df_ap_DM['fulltext'].str.contains('apples|pie|pies|recipe|tart|crumble|bake|vinegar')]
df_ap_DM.drop(yy.index,inplace=True)

In [52]:
df_ap_DM.to_csv('df_ap_DM.csv')

# Datafame Google 

In [53]:
#URL de base 
base_url1 = 'https://www.dailymail.co.uk/home/search.html?offset='
base_url2 = '&size=50&sel=site&searchPhrase=Google&sort=recent&type=article&topic=Google+%28Old%29&topic=Google&days=all'

In [54]:
df_news_go = articles_from_base_url_DM(base_url1, base_url2)

0
1000
2000
3000
4000
5000


In [55]:
df_news_go['url'] = df_news_go['url'].str.lower()

In [56]:
df_go_DM = df_news_go[df_news_go['url'].str.contains('google|android|chrome|gmail|maps|playstore|pixel|waze|youTube|alphabet|mandiant|fitbit|looker|nest|doubleclick|page|brin|pichai|kurian|gafa|gafam|big-tech|consumer-privacy|tech')]

In [58]:
df_go_DM.to_csv('df_go_DM.csv')

# Dataframe Microsoft

In [59]:
#URL de base 
base_url1 = 'https://www.dailymail.co.uk/home/search.html?offset='
base_url2 = '&size=50&sel=site&searchPhrase=Microsoft&sort=recent&type=article&topic=Microsoft&days=all'

In [60]:
df_news_mi = articles_from_base_url_DM(base_url1, base_url2)

0


In [61]:
df_news_mi['url'] = df_news_mi['url'].str.lower()

In [62]:
df_mi_DM = df_news_mi[df_news_mi['url'].str.contains('microsoft|windows|cortana|excel|explorer|office|edge|teams|outlook|powerpoint|skype|surface|word|xbox|linkedIn|github|mojang|gates|nadella|gafa|gafam|big-tech|consumer-privacy|tech')]

In [63]:
df_mi_DM.to_csv('df_mi_DM.csv')

# Dataframe final 

In [64]:
#from google.colab import files
df_final = pd.concat([df_fb_DM, df_am_DM, df_ap_DM, df_go_DM, df_mi_DM]).reset_index()
df_final.drop_duplicates(subset ="url", keep = 'first', inplace=True)
df_final.to_csv('articlesDM.csv') 
#files.download('articlesDM.csv')